## Yule model with melanoma data

PDF of a censured observation $t$ is given by:

              exp(
               p * rho * (-exp(mu*t) + 1) * exp(lambda*t)
               /
               (-exp(lambda * t) + exp(t * (lambda + mu)) + 1)
              )


PDF of a non-censured observation $t$ is given by:

              p * rho
              *
              (-lambda + mu*(exp(lambda*t) - 1) + (lambda + mu)*(-exp(lambda*t) + exp(t*(lambda + mu)) + 1))
              *
              exp(-p*rho*(exp(mu*t) - 1)*exp(lambda*t)/(-exp(lambda*t) + exp(t*(lambda + mu)) + 1))
              /
              (-exp(lambda*t) + exp(t*(lambda + mu)) + 1)**2

With:

              rho = exp(beta0)
              p = inv_logit(beta1 * x)
              lambda = exp(beta2 * x)

### Bayesian Estimation

In [1]:
require(rstan)

db <- read.table("melanoma.txt", TRUE)

t_obs <- db[db$status == 1, "y"]
x1_obs <- db[db$status == 1, "x1"]

t_cen <- db[db$status != 1, "y"]
x1_cen <- db[db$status != 1, "x1"]

n_obs <- length(t_obs)
n_cen <- length(t_cen)

stan_code <- "
data {
  int n_obs;
  int n_cen;
  real t_obs[n_obs];
  real t_cen[n_cen];
  real x1_obs[n_obs];
  real x1_cen[n_cen];
}
parameters {
  real beta0;
  real beta1;
  real beta2;
  real<lower=0> mu;
}
transformed parameters {
  real<lower=0, upper=1> inv_logit_beta1;
  real<lower=0> exp_beta2;
  real<lower=0> rho;

  inv_logit_beta1 = inv_logit(beta1);
  exp_beta2 = exp(beta2);
  rho = exp(beta0);
}
model {
  for (i in 1:n_cen) {
    real t;
    real p;
    real lambda;

    t = t_cen[i];
    if (x1_cen[i] == 1) {
      p = inv_logit_beta1;
      lambda = exp_beta2;
    } else {
      p = 0.5;
      lambda = 1.0;
    }

    target +=
      (exp(mu*t) - 1) *
      p * rho * exp(lambda*t) /
      (exp(lambda*t) - exp(t*(lambda + mu)) - 1);
  }

  for (i in 1:n_obs) {
    real t;
    real p;
    real lambda;

    t = t_obs[i];
    if (x1_obs[i] == 1) {
      p = inv_logit_beta1;
      lambda = exp_beta2;
    } else {
      p = 0.5;
      lambda = 1.0;
    }

    target +=
      (exp(mu*t) - 1) *
      p * rho * exp(lambda*t) /
      (exp(lambda*t) - exp(t*(lambda + mu)) - 1);

    target +=
      log(-lambda + mu*(exp(lambda*t) - 1) + (lambda + mu)*
      (-exp(lambda*t) + exp(t*(lambda + mu)) + 1));

    target += log(p);
    target += log(rho);

    target +=
      - 2 * log(-exp(lambda*t) + exp(t*(lambda + mu)) + 1);
  }

  mu ~ cauchy(0, 2.5);
  beta1 ~ normal(0, 2);
  beta2 ~ normal(0, 2);
}
"

compiled_model <- rstan::stan_model(model_code=stan_code)
capture.output(fit <- rstan::sampling(compiled_model, iter=1e5, chains=3, init_r=.01, cores=3, refresh=100,
                                      control=list(adapt_delta=0.99, max_treedepth=20)), file='stan_log')

Loading required package: rstan
Loading required package: ggplot2
Loading required package: StanHeaders
rstan (Version 2.16.2, packaged: 2017-07-03 09:24:58 UTC, GitRev: 2e1f913d3ca3)
For execution on a local, multicore CPU with excess RAM we recommend calling
rstan_options(auto_write = TRUE)
options(mc.cores = parallel::detectCores())


In [2]:
fit

Inference for Stan model: 15775541c16dc9c5cc7dfa71e5cd1c26.
3 chains, each with iter=1e+05; warmup=50000; thin=1; 
post-warmup draws per chain=50000, total post-warmup draws=150000.

                   mean se_mean   sd    2.5%     25%     50%     75%   97.5%
beta0              0.17    0.00 0.11   -0.05    0.09    0.17    0.24    0.38
beta1              0.54    0.00 0.56   -0.23    0.18    0.45    0.79    1.89
beta2             -0.48    0.00 0.23   -1.02   -0.61   -0.45   -0.32   -0.12
mu                 0.10    0.00 0.02    0.07    0.09    0.10    0.11    0.13
inv_logit_beta1    0.62    0.00 0.11    0.44    0.55    0.61    0.69    0.87
exp_beta2          0.63    0.00 0.13    0.36    0.55    0.64    0.72    0.89
rho                1.19    0.00 0.13    0.95    1.10    1.19    1.28    1.46
lp__            -534.57    0.01 1.61 -538.69 -535.33 -534.19 -533.39 -532.60
                n_eff Rhat
beta0           61687    1
beta1           38650    1
beta2           48471    1
mu              

In [3]:
res <- as.matrix(fit)

beta0 <- res[, 'beta0']
beta1 <- res[, 'beta1']
rho <- res[, 'rho']
log_inv_beta1 <- res[, 'inv_logit_beta1']
exp_beta2 <- res[, 'exp_beta2']
mu <- res[, 'mu']

concat <- cbind(beta0, beta1, rho, log_inv_beta1, exp_beta2, mu)
point_estimates <- colMeans(concat)
results <- cbind(estimate=point_estimates, t(apply(concat, 2, function(x){quantile(x, c(0.025, 0.975))})))
results

,estimate,2.5%,97.5%
beta0,0.16856514,-0.05354433,0.3811647
beta1,0.54486955,-0.23289689,1.8895071
rho,1.19087306,0.94786392,1.4639887
log_inv_beta1,0.62244840,0.44203754,0.8686993
exp_beta2,0.63333926,0.35887866,0.8880192
mu,0.09859869,0.07171876,0.1308422


### Optimization (MAP)

In [4]:
optim <- rstan::optimizing(compiled_model)

Initial log joint probability = -1282.39
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


In [5]:
optim

$par
          beta0           beta1           beta2              mu inv_logit_beta1 
     0.19760288      0.28466244     -0.37022931      0.09356358      0.57068891 
      exp_beta2             rho 
     0.69057596      1.21847842 

$value
[1] -529.9727

$return_code
[1] 0

In [10]:
# PDF of the survival function against KM

long_time_sample <- function(lambda, mu, rho, p) {

  yule_start <- function(lambda_, d) {
    self = list()
    self$lambda_ = lambda_
    self$upsilon = rexp(d, lambda_)
    return(self)
  }

  yule_get_next <- function(self) {
    current_min_index = which.min(self$upsilon)
    current_min = self$upsilon[current_min_index]

    self$upsilon[current_min_index] = current_min + rexp(1, self$lambda_)
    self$upsilon[length(self$upsilon) + 1] = current_min + rexp(1, self$lambda_)

    self$current_min = current_min
    return(self)
  }

  m <- rpois(1, rho)
  d <- rbinom(1, m, p)

  if (d == 0) {
    return(Inf)
  }

  v <- d

  process <- yule_start(lambda, d)
  new_t <- 0
  y <- rexp(d, mu)

  while (TRUE) {
    process <- yule_get_next(process)
    new_t <- process$current_min

    # A previous cell got promoted
    if (min(y) < new_t)
      return(min(y))

    new_y <- rexp(1, mu)

    # This cell got promoted
    if (new_y < new_t)
      return(new_t)

    y <- c(y, new_y)
    v <- v+1
  }
}

# Em seguida escolhemos os valores dos parametros:

lambda <- c(1, point_estimates["exp_beta2"])
mu <- point_estimates["mu"]
p <- c(0.5, point_estimates["log_inv_beta1"])
rho <- point_estimates["rho"]

observations <- censoring_ind <- list()
for (i in 1:length(p)) {
  observations[[i]] <- replicate(1e4, long_time_sample(lambda[i], mu, rho, p[i]))
  censoring_ind[[i]] <- (observations[[i]] < Inf)
}

colors <- c("blue", "red", "forestgreen", "black")

#Por fim, a sobrevivencia empirica (i.e.: Kaplan-Meier)
require(survival)
pdf("surv.pdf",width=7,height=5)

plot(survfit(Surv(db$y[db$x1 == 0], db$status[db$x1 == 0])~1, conf.type="none"), lty=3, lwd=2.2,
     col=colors[3], ylim=c(0.45, 1), xlim=c(0,7), xlab="Time", ylab="Survival function")
lines(survfit(Surv(db$y[db$x1 == 1], db$status[db$x1 == 1])~1, conf.type="none"), lty=4, lwd=2.2, col=colors[4])
lines(survfit(Surv(observations[[1]], censoring_ind[[1]])~1, conf.type="none"), lty=1, lwd=2.2, col=colors[1])
lines(survfit(Surv(observations[[2]], censoring_ind[[2]])~1, conf.type="none"), lty=2, lwd=2.2, col=colors[2])

legend_names <- c(
                  "Mean posterior for group without treatment",
                  "Mean posterior for group with treatment",
                  "KM for group without treatment",
                  "KM for group with treatment"
                 )
legend("topright", legend=legend_names, cex=0.8, col=colors, inset=.001,
      lty=1:4, lwd=2.2, bty="n")
dev.off()

png 
  2

In [9]:
pdf("posterior.pdf", width=7, height=5)

colors <- c("blue", "red", "brown", "black", "yellow", "green")

plot(density(dpois(0, res[,"rho"] * 0.5), "SJ"), col=colors[1], lty=1, lwd=2.1,
      ylim=c(.05, 26), xlim=c(0,1.6), main="Posterior density curves", xlab='', log='y') #cure for non-treatment
lines(density(dpois(0, res[,"rho"] * res[,"inv_logit_beta1"]), "SJ"), col=colors[2], lty=2, lwd=2.1) #cure for treatment

lines(density(res[,"inv_logit_beta1"], "SJ"), col=colors[3], lty=3, lwd=2.1)
lines(density(res[,"exp_beta2"], "SJ"), col=colors[4], lty=1, lwd=2.8) #lambda for treatment

lines(density(res[,"rho"], "SJ"), col=colors[5], lty=2, lwd=2.8)
lines(density(res[,"mu"], "SJ"), col=colors[6], lty=3, lwd=2.8)

legend_names <- c(
                  "Cure rate without treatment",
                  "Cure rate with treatment", 
    
                  expression(paste(p, " for treatment group")),
                  expression(paste(lambda, " for treatment group")),

                  expression(rho),
                  expression(mu)
                 )
legend("topright", legend=legend_names, cex=0.8, col=colors, inset=.001,
       lty=1:3, lwd=c(1.1, 1.1, 1.1, 1.8, 1.8, 1.8) + 1,
       bty="n")

dev.off()

png 
  2